In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
def test4(df):
    temperature_test = (df['Temperature'] >= 5) & (df['Temperature'] <= 60)
    humidity_test = (df['RH'] >= 0) & (df['RH'] <= 100)
    test4_passed = temperature_test & humidity_test
    return test4_passed

In [ ]:
files = os.listdir("rawdata")
files.sort()

for file in files:
    history_file = file.split("-")[0] + file.split("-")[1] + ".CSV"
    df_weatherdata = pd.read_csv(f"weatherdata/{history_file}", parse_dates=['valid_time_gmt'], index_col=['valid_time_gmt'])
    df_weatherdata = df_weatherdata.loc[:, ["temp", "rh"]]
    df_weatherdata = df_weatherdata.resample("30T", label="left").mean()
    df_weatherdata.fillna(method='ffill', inplace=True)

    df_weatherindex = pd.DataFrame(df_weatherdata.index, index=df_weatherdata.index)
    df_weatherdata["date"] = df_weatherindex["valid_time_gmt"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df_weatherdata["hr"] = df_weatherindex["valid_time_gmt"].apply(lambda x: x.strftime("%H"))
    df_weatherdata["min"] = df_weatherindex["valid_time_gmt"].apply(lambda x: x.strftime("%M"))

    df_rawdata = df_rawdata = pd.read_csv(f"rawdata/{file}", header=0, parse_dates=['timestamp'], index_col=0)

    df_rawdataindex = pd.DataFrame(df_rawdata.index, index=df_rawdata.index)
    df_rawdata["hr"] = df_rawdataindex["timestamp"].apply(lambda x: x.strftime("%H"))
    df_rawdata["min"] = df_rawdataindex["timestamp"].apply(lambda x: x.strftime("%M"))
    df_rawdata["min"] = df_rawdata["min"].apply(lambda x: "00" if int(x) < 30 else "30")

    df_merged = pd.merge(df_rawdata, df_weatherdata, on=["date", "hr", "min"], how="left")
    df_merged.index = df_rawdata.index
    df_merged.loc[test4(df_rawdata) == False, "Temperature"] = df_merged.loc[test4(df_rawdata) == False, "temp"]
    df_merged.loc[test4(df_rawdata) == False, "RH"] = df_merged.loc[test4(df_rawdata) == False, "rh"]
    df_merged.drop(["hr", "min", "temp", "rh"], axis=1, inplace=True)

    df_merged.to_csv(f"correcteddata/{file}", index=True, header=True)